In [370]:
import pandas as pd
import pickle
from tqdm import tqdm
import requests
from bs4 import BeautifulSoup
import time

import warnings
warnings.filterwarnings('ignore')

In [6]:
pd.set_option('display.max_columns', 100)

In [242]:
def getAndParseUrl(url):
    results = requests.get(url)
    time.sleep(1)
    soup = BeautifulSoup(results.text, 'html.parser')
    return soup

In [335]:
def get_stats_and_positions():
    df = pd.DataFrame()
    for i in tqdm(range(1950,2020,1)):
        table1 = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{i}_totals.html')
        table2 = pd.read_html(f'https://www.basketball-reference.com/leagues/NBA_{i}_advanced.html')
        
        soup1 = getAndParseUrl(f'https://www.basketball-reference.com/leagues/NBA_{i}_totals.html')
        soup2 = getAndParseUrl(f'https://www.basketball-reference.com/leagues/NBA_{i}_advanced.html')
        
        pid1 = [tag.attrs['data-append-csv'] for tag in [td for td in soup1.select('td') if td.attrs['data-stat']=='player']]
        pid2 = [tag.attrs['data-append-csv'] for tag in [td for td in soup2.select('td') if td.attrs['data-stat']=='player']]
        
        table1[0] = table1[0][table1[0].Player != 'Player']
        table2[0] = table2[0][table2[0].Player != 'Player']
        
        table1[0]['pid'] = pid1
        table2[0]['pid'] = pid2
        
        table2[0] = table2[0].drop(['Player', 'Rk', 'Pos', 'Age', 'G', 'MP', 'Unnamed: 19', 'Unnamed: 24'], axis=1)
        
        stats_df = table1[0].merge(table2[0], on=['pid', 'Tm'], how='left')
        stats_df['season'] = i
        
        df = pd.concat([df,stats_df])
        
        df.to_pickle('df_1950to2019_stats.pickle')
        
    return df[df.Player != 'Player']

In [336]:
df = get_stats_and_positions()

100%|██████████| 70/70 [10:06<00:00, 11.49s/it]


In [103]:
df.columns = [col.lower().replace('%','_pct') for col in df.columns]

In [105]:
# https://help.yahoo.com/kb/fantasy-basketball/default-league-settings-fantasy-basketball-sln6919.html
# yahoo's fantasy scoring system
pts = 1
reb = 1.2
ast = 1.5
blk = 3
stl = 3
tov = -1

In [106]:
df.player = [player.strip('*') for player in df.player]

In [378]:
df.to_pickle('df_1950to2019_stats.pickle')